# Parsing Wiki Data
`MMV | 12/4 | w266 Final Project: Crosslingual Word Embeddings`   


The code in this notebook builds on the helper functions provided in the TensorFlow Word2Vec tutorial to develop a set of data handling functions for use with the data relevant to Duong et al's paper. Ideally I'll develop a scalable solution for tokenizing, prepending language indicators (eg. `en_`) and extracting sentences in two langauges to create traning data that includes sentences from two languages. I also hope to develop a batch iterator modeled after the one in A4. Depending on the available tools I may end up needing to look at using a distributed system (Spark?) for preprocessing the English corpus which is ~ 9GB.

# Notebook Set-up

In [1]:
# general imports
import os
import sys  
import numpy as np
import matplotlib.pyplot as plt
from __future__ import print_function
# tell matplotlib not to open a new window
%matplotlib inline

In [2]:
# filepaths
BASE = '/home/mmillervedam/Data'
FPATH_EN = BASE + '/test/wiki_en_10K.txt' # first 10000 lines from wiki dump
FPATH_ES = BASE + '/test/wiki_es_10K.txt' # first 10000 lines from wiki dump
FULL_EN = BASE + '/en/full.txt'
FULL_ES = BASE + '/es/full.txt'
DPATH = '/home/mmillervedam/ProjectRepo/XlingualEmb/data/dicts/en.es.panlex.all.processed'

__`NOTE:`__ First time you run this on a new machine you'll need to make sure you've installed [polyglot](http://polyglot.readthedocs.io/en/latest/Installation.html):
```
sudo apt-get install libicu-dev
pip install polyglot
```

In [8]:
import polyglot

# Tokenizing

In [12]:
from polyglot.detect import Detector

ImportError: /home/mmillervedam/anaconda2/lib/python2.7/site-packages/_icu.so: undefined symbol: _ZTIN6icu_5514LEFontInstanceE

In [11]:
from polyglot.text import *

ImportError: /home/mmillervedam/anaconda2/lib/python2.7/site-packages/_icu.so: undefined symbol: _ZTIN6icu_5514LEFontInstanceE

In [7]:
blob = "[[12]] Anarchism is often defined as a political philosophy which holds the state to be undesirable , unnecessary , or harmful ."

In [ ]:
# Helper function
def read_data(filename):
    """
    Extract the file as a list of words.
    NOTE: this is modified from original function in TF  
    tutorialwhich expected a zipped input file.
    """
    with open(filename) as f:
        data = tf.compat.as_str(f.read()).split()
    return data